<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/BDSN_SparkSQL_OLAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-05-03 09:37:17.332039+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


In [2]:
#import numpy as np
import pandas as pd
#import pyarrow.csv as pv
#import pyarrow.parquet as pq

#SuperStore Data
Super Store Order Data https://drive.google.com/file/d/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ/view?usp=sharing <br>
Original Data from [Tableau](https://community.tableau.com/docs/DOC-1236)

In [3]:
#!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'
#!gdown https://drive.google.com/uc?id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
#!wget -q -O Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'
!wget -q -O Orders.zip 'https://github.com/Praxis-QR/PythonForManagers/raw/main/Data/SuperStoreSalesKaggle.zip'

#Pandas - Benchmark

##Data Cleaning with Pandas

In [4]:
df = pd.read_csv('Orders.zip', delimiter = ',')
df.info()

# order_date, ship_date, sales have been incorrectly formatted as string
# year has been incorrectly formatted as integer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        51290 non-null  object 
 1   order_date      51290 non-null  object 
 2   ship_date       51290 non-null  object 
 3   ship_mode       51290 non-null  object 
 4   customer_name   51290 non-null  object 
 5   segment         51290 non-null  object 
 6   state           51290 non-null  object 
 7   country         51290 non-null  object 
 8   market          51290 non-null  object 
 9   region          51290 non-null  object 
 10  product_id      51290 non-null  object 
 11  category        51290 non-null  object 
 12  sub_category    51290 non-null  object 
 13  product_name    51290 non-null  object 
 14  sales           51290 non-null  object 
 15  quantity        51290 non-null  int64  
 16  discount        51290 non-null  float64
 17  profit          51290 non-null 

### Sales, year needs to be fixed

In [5]:
# df.sales has been automatically interpreted as string because of the presence of commma,
# so replace comma and convert to numeric
# convert year to string
#

df['sales'] = df['sales'].str.replace(',', '')    # replace , with blank
df['sales'] = pd.to_numeric(df['sales'])          # convert column to numeric
df['year'] = df['year'].astype(str)          # convert column to string

In [ ]:
#df.info()

###Date Problems

In [6]:
# what is the first and last order date ?
# we have four years of data, but date format changes ...
#
#df['order_date'].describe
#df['ship_date'].describe
df['order_date'].head(), df['order_date'].tail()

(0    1/1/2011
 1    1/1/2011
 2    1/1/2011
 3    1/1/2011
 4    1/1/2011
 Name: order_date, dtype: object,
 51285    31-12-2014
 51286    31-12-2014
 51287    31-12-2014
 51288    31-12-2014
 51289    31-12-2014
 Name: order_date, dtype: object)

In [7]:
# Some of the dates are string like dd/mm/yyyy while others are dd-mm-yyyy
# this will cause problems in converting string to real dates
# So we move all dd-mm-yyyy to dd/mm/yyyy -- could have been the other way round also
#

df['order_date'] = df['order_date'].str.replace('-','/')
df['ship_date'] = df['ship_date'].str.replace('-','/')

# convert all dates to dd/mm/yyyy format
#
df['ship_date'] = pd.to_datetime(df['ship_date'],format='%d/%m/%Y')
df['order_date'] = pd.to_datetime(df['order_date'],format='%d/%m/%Y')
#df['order_date'].describe

In [ ]:
# product_name contains comma, doesnt really matter
#df['product_name'] = df['product_name'].str.replace('"','')


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        51290 non-null  object        
 1   order_date      51290 non-null  datetime64[ns]
 2   ship_date       51290 non-null  datetime64[ns]
 3   ship_mode       51290 non-null  object        
 4   customer_name   51290 non-null  object        
 5   segment         51290 non-null  object        
 6   state           51290 non-null  object        
 7   country         51290 non-null  object        
 8   market          51290 non-null  object        
 9   region          51290 non-null  object        
 10  product_id      51290 non-null  object        
 11  category        51290 non-null  object        
 12  sub_category    51290 non-null  object        
 13  product_name    51290 non-null  object        
 14  sales           51290 non-null  int64         
 15  qu

In [9]:
df.to_csv('Orders.csv', index=False)

##Pandas Pivot

In [10]:
pd.pivot_table(df, index = 'segment', columns = 'year', values = 'sales', aggfunc = 'sum', margins = True)

year,2011,2012,2013,2014,All
segment,,,,,
Consumer,1173701,1463804,1729284,2141352,6508141
Corporate,691685,774450,1065033,1293640,3824808
Home Office,394125,439239,611543,865049,2309956
All,2259511,2677493,3405860,4300041,12642905


In [11]:
pd.pivot_table(df, index = 'segment', columns = 'category', values = 'quantity', aggfunc = 'sum', margins = True)

category,Furniture,Office Supplies,Technology,All
segment,,,,
Consumer,18002,56025,18130,92157
Corporate,10735,32236,10594,53565
Home Office,6217,19921,6452,32590
All,34954,108182,35176,178312


#Spark
https://davidadrian.cc/python-hadoop/

In [12]:
!pip3 -q install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [13]:
from pyspark.sql import SparkSession

#https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import regexp_replace

In [15]:
#spark9 = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
spark9 = SparkSession.\
    builder.\
    master("local[*]").\
    config('spark.ui.port', '4050').\
    appName('Spark9').\
    getOrCreate()

spark9

In [16]:
!unzip Orders.zip

Archive:  Orders.zip
  inflating: SuperStoreOrders.csv    


## Data Cleaning with Spark

In [17]:
!free

               total        used        free      shared  buff/cache   available
Mem:        13290480     1211896     4336836        2052     7741748    11751124
Swap:              0           0           0


In [18]:
#Inferred INCORRECTLY, from CSV file

sdf = spark9.read.csv('SuperStoreOrders.csv', inferSchema=True,header = True, quote = '"', escape = '"')
#sdf9A = spark9.read.csv('Orders3.csv', inferSchema=True,header = True, quote = '"')
#sdf9A = spark9.read.csv('SuperStoreOrders.csv', inferSchema=True,header = True)

sdf.dtypes

[('order_id', 'string'),
 ('order_date', 'string'),
 ('ship_date', 'string'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'string'),
 ('quantity', 'int'),
 ('discount', 'double'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'int')]

In [19]:
!free

               total        used        free      shared  buff/cache   available
Mem:        13290480     1246500     4294956        2060     7749024    11716512
Swap:              0           0           0


In [ ]:
#sdf9A.filter(df['sales'].rlike('[\!\@\$\^\&\-\_\;\:\?\.\#\*]')).count()
#sdf9A.filter(df['sales'].like('[\!\@\$\^\&\-\_\;\:\?\.\#\*\,]')).show()
#sdf.filter(col('sales').contains(",")).show()

In [20]:
#https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
#from pyspark.sql.functions import col, to_date
#from pyspark.sql.functions import regexp_replace
sdf = sdf.\
        withColumn('order_date',regexp_replace('order_date','/','-')).\
        withColumn('ship_date',regexp_replace('order_date','/','-')).\
        withColumn('order_date',to_date(col('order_date'), "d-M-y")).\
        withColumn('ship_date',to_date(col('ship_date'), "d-M-y")).\
        withColumn('sales',regexp_replace('sales',',','')).\
        withColumn('sales',col('sales').cast('int')).\
        withColumn('quantity',col('quantity').cast('int')).\
        withColumn('year',col('year').cast('string'))

sdf.dtypes

[('order_id', 'string'),
 ('order_date', 'date'),
 ('ship_date', 'date'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'int'),
 ('quantity', 'int'),
 ('discount', 'double'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'string')]

In [ ]:
#sdf9A.filter(df['sales'].rlike('[\!\@\$\^\&\-\_\;\:\?\.\#\*]')).count()
#sdf9A.filter(df['sales'].like('[\!\@\$\^\&\-\_\;\:\?\.\#\*\,]')).show()
#sdf.filter(col('sales').contains(",")).show()
#sdf.filter(col('product_name').contains("\"")).show()

In [ ]:
#sdf9A.show()

## Pivots - check with Panda pivots

In [21]:
sdf.groupBy('segment').pivot('year').sum('sales').show()

+-----------+-------+-------+-------+-------+
|    segment|   2011|   2012|   2013|   2014|
+-----------+-------+-------+-------+-------+
|   Consumer|1173701|1463804|1729284|2141352|
|Home Office| 394125| 439239| 611543| 865049|
|  Corporate| 691685| 774450|1065033|1293640|
+-----------+-------+-------+-------+-------+



In [22]:
sdf.groupBy('segment').pivot('category').sum('quantity').show()

+-----------+---------+---------------+----------+
|    segment|Furniture|Office Supplies|Technology|
+-----------+---------+---------------+----------+
|   Consumer|    18002|          56025|     18130|
|Home Office|     6217|          19921|      6452|
|  Corporate|    10735|          32236|     10594|
+-----------+---------+---------------+----------+



In [ ]:
# remove , and " from data
#import pandas as pd
#p_df = pd.read_csv('SSdatafile.csv')
#p_df = pd.read_csv('datafile.csv')
#p_df2 = p_df.replace(',', ' ', regex=True)
#p_df3 = p_df2.replace('"', '', regex=True)
#p_df3.to_csv('SSdatafile2.csv',index=False,sep=',')
#!head SSdatafile2.csv

#Spark SQL

##Create Spark Hive Tables

In [23]:
spark9.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [ ]:
#sdf.dtypes

In [24]:
colSet = ['segment','category','region','year','sales','quantity','profit']
sdf = sdf.select(colSet)
sdf.show()

+-----------+---------------+--------------+----+-----+--------+---------+
|    segment|       category|        region|year|sales|quantity|   profit|
+-----------+---------------+--------------+----+-----+--------+---------+
|   Consumer|Office Supplies|        Africa|2011|  408|       2|   106.14|
|   Consumer|Office Supplies|       Oceania|2011|  120|       3|   36.036|
|   Consumer|Office Supplies|          EMEA|2011|   66|       4|    29.64|
|Home Office|Office Supplies|         North|2011|   45|       3|  -26.055|
|   Consumer|      Furniture|       Oceania|2011|  114|       5|    37.77|
|   Consumer|Office Supplies|       Oceania|2011|   55|       2|   15.342|
|   Consumer|     Technology|        Canada|2011|  314|       1|     3.12|
|  Corporate|Office Supplies|       Oceania|2011|  276|       1|  110.412|
|   Consumer|     Technology|       Oceania|2011|  912|       4| -319.464|
|  Corporate|      Furniture|          EMEA|2011|  667|       4|   253.32|
|   Consumer|Office Suppl

In [25]:
!free

               total        used        free      shared  buff/cache   available
Mem:        13290480     1355140     4183668        2108     7751672    11607824
Swap:              0           0           0


In [26]:
#air_quality_sdf.createOrReplaceTempView("air_quality_sdf")
sdf.createOrReplaceTempView("tFact")
spark9.sql("describe tFact").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
| segment|   string|   NULL|
|category|   string|   NULL|
|  region|   string|   NULL|
|    year|   string|   NULL|
|   sales|      int|   NULL|
|quantity|      int|   NULL|
|  profit|   double|   NULL|
+--------+---------+-------+



In [27]:
!free

               total        used        free      shared  buff/cache   available
Mem:        13290480     1350464     4188300        2116     7751716    11612492
Swap:              0           0           0


In [ ]:
spark9.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |    tfact|       true|
+---------+---------+-----------+



In [28]:
spark9.sql('select * from tFact limit 5').show()

+-----------+---------------+-------+----+-----+--------+-------+
|    segment|       category| region|year|sales|quantity| profit|
+-----------+---------------+-------+----+-----+--------+-------+
|   Consumer|Office Supplies| Africa|2011|  408|       2| 106.14|
|   Consumer|Office Supplies|Oceania|2011|  120|       3| 36.036|
|   Consumer|Office Supplies|   EMEA|2011|   66|       4|  29.64|
|Home Office|Office Supplies|  North|2011|   45|       3|-26.055|
|   Consumer|      Furniture|Oceania|2011|  114|       5|  37.77|
+-----------+---------------+-------+----+-----+--------+-------+



In [29]:
sql_drop_table = """
drop table if exists praxisdb.fact43
"""

sql_drop_database = """
drop database if exists praxisdb cascade
"""

sql_create_database = """
create database if not exists praxisdb
location '/content/praxisdb/'
"""

sql_create_table = """
create table if not exists praxisdb.fact43
using parquet
as select  * from tFact
"""

#as select to_timestamp(date) as date_parsed, *

print("dropping database...")
result_drop_db = spark9.sql(sql_drop_database)

print("creating database...")
result_create_db = spark9.sql(sql_create_database)

print("dropping table...")
result_droptable = spark9.sql(sql_drop_table)

print("creating table...")
result_create_table = spark9.sql(sql_create_table)

dropping database...
creating database...
dropping table...
creating table...


In [30]:
spark9.sql("use praxisdb")
spark9.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
| praxisdb|   fact43|      false|
|         |    tfact|       true|
+---------+---------+-----------+



In [31]:
spark9.sql("select * from praxisdb.fact43").show(5)

+-----------+---------------+-------+----+-----+--------+-------+
|    segment|       category| region|year|sales|quantity| profit|
+-----------+---------------+-------+----+-----+--------+-------+
|   Consumer|Office Supplies| Africa|2011|  408|       2| 106.14|
|   Consumer|Office Supplies|Oceania|2011|  120|       3| 36.036|
|   Consumer|Office Supplies|   EMEA|2011|   66|       4|  29.64|
|Home Office|Office Supplies|  North|2011|   45|       3|-26.055|
|   Consumer|      Furniture|Oceania|2011|  114|       5|  37.77|
+-----------+---------------+-------+----+-----+--------+-------+
only showing top 5 rows



In [32]:
spark9.catalog.dropTempView("tFact")

True

In [37]:
!free

               total        used        free      shared  buff/cache   available
Mem:        13290480     1440512     6085540        2656     5764428    11521904
Swap:              0           0           0


In [34]:
# Assuming df is the DataFrame you want to delete
sdf.unpersist()
del sdf

In [38]:
!free

               total        used        free      shared  buff/cache   available
Mem:        13290480     1450856     6075212        2656     5764412    11511560
Swap:              0           0           0


In [36]:
spark9.sql("use praxisdb")
spark9.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
| praxisdb|   fact43|      false|
+---------+---------+-----------+



#OLAP

In [39]:
spark9.sql("select * from praxisdb.fact43").show()

+-----------+---------------+--------------+----+-----+--------+---------+
|    segment|       category|        region|year|sales|quantity|   profit|
+-----------+---------------+--------------+----+-----+--------+---------+
|   Consumer|Office Supplies|        Africa|2011|  408|       2|   106.14|
|   Consumer|Office Supplies|       Oceania|2011|  120|       3|   36.036|
|   Consumer|Office Supplies|          EMEA|2011|   66|       4|    29.64|
|Home Office|Office Supplies|         North|2011|   45|       3|  -26.055|
|   Consumer|      Furniture|       Oceania|2011|  114|       5|    37.77|
|   Consumer|Office Supplies|       Oceania|2011|   55|       2|   15.342|
|   Consumer|     Technology|        Canada|2011|  314|       1|     3.12|
|  Corporate|Office Supplies|       Oceania|2011|  276|       1|  110.412|
|   Consumer|     Technology|       Oceania|2011|  912|       4| -319.464|
|  Corporate|      Furniture|          EMEA|2011|  667|       4|   253.32|
|   Consumer|Office Suppl

##Group By

In [41]:
spark9.sql("\
select segment, sum(quantity) tot_quantity, sum(sales) tot_sales \
from fact43 \
group by segment \
order by segment \
").show()

+-----------+------------+---------+
|    segment|tot_quantity|tot_sales|
+-----------+------------+---------+
|   Consumer|       92157|  6508141|
|  Corporate|       53565|  3824808|
|Home Office|       32590|  2309956|
+-----------+------------+---------+



In [42]:
spark9.sql("\
select segment, year, sum(quantity) tot_quantity, sum(sales) tot_sales \
from fact43 \
group by segment, year \
order by segment, year \
").show()

+-----------+----+------------+---------+
|    segment|year|tot_quantity|tot_sales|
+-----------+----+------------+---------+
|   Consumer|2011|       16455|  1173701|
|   Consumer|2012|       20259|  1463804|
|   Consumer|2013|       24601|  1729284|
|   Consumer|2014|       30842|  2141352|
|  Corporate|2011|        9663|   691685|
|  Corporate|2012|       11296|   774450|
|  Corporate|2013|       14685|  1065033|
|  Corporate|2014|       17921|  1293640|
|Home Office|2011|        5325|   394125|
|Home Office|2012|        6556|   439239|
|Home Office|2013|        8850|   611543|
|Home Office|2014|       11859|   865049|
+-----------+----+------------+---------+



In [43]:
spark9.sql("\
select segment, year, sum(quantity) tot_quantity, sum(sales) tot_sales \
from fact43 \
group by segment, year \
union all \
select segment, NULL, sum(quantity) tot_quantity, sum(sales) tot_sales \
from fact43 \
group by segment \
union all \
select NULL, year, sum(quantity) tot_quantity, sum(sales) tot_sales \
from fact43 \
group by year \
union all \
select null, null, sum(quantity) tot_quantity, sum(sales) tot_sales \
from fact43 \
").show()

+-----------+----+------------+---------+
|    segment|year|tot_quantity|tot_sales|
+-----------+----+------------+---------+
|  Corporate|2011|        9663|   691685|
|Home Office|2012|        6556|   439239|
|   Consumer|2014|       30842|  2141352|
|Home Office|2013|        8850|   611543|
|  Corporate|2012|       11296|   774450|
|   Consumer|2011|       16455|  1173701|
|Home Office|2011|        5325|   394125|
|Home Office|2014|       11859|   865049|
|   Consumer|2012|       20259|  1463804|
|  Corporate|2013|       14685|  1065033|
|   Consumer|2013|       24601|  1729284|
|  Corporate|2014|       17921|  1293640|
|   Consumer|NULL|       92157|  6508141|
|Home Office|NULL|       32590|  2309956|
|  Corporate|NULL|       53565|  3824808|
|       NULL|2012|       38111|  2677493|
|       NULL|2014|       60622|  4300041|
|       NULL|2013|       48136|  3405860|
|       NULL|2011|       31443|  2259511|
|       NULL|NULL|      178312| 12642905|
+-----------+----+------------+---

##Grouping Sets

In [44]:
spark9.sql("\
select segment, year, sum(quantity) tot_quantity, sum(sales) tot_sales \
from fact43 \
group by \
    grouping sets( \
        (segment, year), \
        (segment),\
        (year), \
        () \
    ) \
order by segment, year \
").show()

+-----------+----+------------+---------+
|    segment|year|tot_quantity|tot_sales|
+-----------+----+------------+---------+
|       NULL|NULL|      178312| 12642905|
|       NULL|2011|       31443|  2259511|
|       NULL|2012|       38111|  2677493|
|       NULL|2013|       48136|  3405860|
|       NULL|2014|       60622|  4300041|
|   Consumer|NULL|       92157|  6508141|
|   Consumer|2011|       16455|  1173701|
|   Consumer|2012|       20259|  1463804|
|   Consumer|2013|       24601|  1729284|
|   Consumer|2014|       30842|  2141352|
|  Corporate|NULL|       53565|  3824808|
|  Corporate|2011|        9663|   691685|
|  Corporate|2012|       11296|   774450|
|  Corporate|2013|       14685|  1065033|
|  Corporate|2014|       17921|  1293640|
|Home Office|NULL|       32590|  2309956|
|Home Office|2011|        5325|   394125|
|Home Office|2012|        6556|   439239|
|Home Office|2013|        8850|   611543|
|Home Office|2014|       11859|   865049|
+-----------+----+------------+---

##Roll Up

In [45]:
spark9.sql("\
select \
ifnull(segment,'all segment') _segment,\
ifnull(year,'all year') _year,\
sum(quantity) tot_quantity, sum(sales) tot_sales \
from praxisdb.fact43 \
group by rollup(segment, year) \
order by _segment, _year \
").show()

+-----------+--------+------------+---------+
|   _segment|   _year|tot_quantity|tot_sales|
+-----------+--------+------------+---------+
|   Consumer|    2011|       16455|  1173701|
|   Consumer|    2012|       20259|  1463804|
|   Consumer|    2013|       24601|  1729284|
|   Consumer|    2014|       30842|  2141352|
|   Consumer|all year|       92157|  6508141|
|  Corporate|    2011|        9663|   691685|
|  Corporate|    2012|       11296|   774450|
|  Corporate|    2013|       14685|  1065033|
|  Corporate|    2014|       17921|  1293640|
|  Corporate|all year|       53565|  3824808|
|Home Office|    2011|        5325|   394125|
|Home Office|    2012|        6556|   439239|
|Home Office|    2013|        8850|   611543|
|Home Office|    2014|       11859|   865049|
|Home Office|all year|       32590|  2309956|
|all segment|all year|      178312| 12642905|
+-----------+--------+------------+---------+



In [46]:
spark9.sql("\
select segment, year, sum(quantity) tot_quantity, sum(sales) tot_sales \
from fact43 \
group by rollup(segment, year) \
order by segment, year \
").show()

+-----------+----+------------+---------+
|    segment|year|tot_quantity|tot_sales|
+-----------+----+------------+---------+
|       NULL|NULL|      178312| 12642905|
|   Consumer|NULL|       92157|  6508141|
|   Consumer|2011|       16455|  1173701|
|   Consumer|2012|       20259|  1463804|
|   Consumer|2013|       24601|  1729284|
|   Consumer|2014|       30842|  2141352|
|  Corporate|NULL|       53565|  3824808|
|  Corporate|2011|        9663|   691685|
|  Corporate|2012|       11296|   774450|
|  Corporate|2013|       14685|  1065033|
|  Corporate|2014|       17921|  1293640|
|Home Office|NULL|       32590|  2309956|
|Home Office|2011|        5325|   394125|
|Home Office|2012|        6556|   439239|
|Home Office|2013|        8850|   611543|
|Home Office|2014|       11859|   865049|
+-----------+----+------------+---------+



##CUBE

In [47]:
spark9.sql("\
select segment, year, sum(quantity) tot_quantity, sum(sales) tot_sales \
from praxisdb.fact43 \
group by cube(segment, year) \
order by segment, year \
").show()

+-----------+----+------------+---------+
|    segment|year|tot_quantity|tot_sales|
+-----------+----+------------+---------+
|       NULL|NULL|      178312| 12642905|
|       NULL|2011|       31443|  2259511|
|       NULL|2012|       38111|  2677493|
|       NULL|2013|       48136|  3405860|
|       NULL|2014|       60622|  4300041|
|   Consumer|NULL|       92157|  6508141|
|   Consumer|2011|       16455|  1173701|
|   Consumer|2012|       20259|  1463804|
|   Consumer|2013|       24601|  1729284|
|   Consumer|2014|       30842|  2141352|
|  Corporate|NULL|       53565|  3824808|
|  Corporate|2011|        9663|   691685|
|  Corporate|2012|       11296|   774450|
|  Corporate|2013|       14685|  1065033|
|  Corporate|2014|       17921|  1293640|
|Home Office|NULL|       32590|  2309956|
|Home Office|2011|        5325|   394125|
|Home Office|2012|        6556|   439239|
|Home Office|2013|        8850|   611543|
|Home Office|2014|       11859|   865049|
+-----------+----+------------+---

##CrossTab

In [48]:
CrossTab2 = spark9.sql("""
 SELECT
  IFNULL(Segment,'#Totals') Dims,
  Africa,North_Asia,Central_Asia,Caribbean,
  Oceania,Canada,EMEA,SE_Asia,
  Central,East,West,South,
  Africa +North_Asia + Central_Asia+ Caribbean + Oceania + Canada+EMEA + SE_Asia + Central + East + West + South Totals
FROM (
SELECT
        Segment,
        SUM(IF(Region='Africa',Sales,0)) As Africa,
        SUM(IF(Region='North Asia',Sales,0)) As North_Asia,
        SUM(IF(Region='Central Asia',Sales,0)) As Central_Asia,
        SUM(IF(Region='Caribbean',Sales,0)) As Caribbean,
        SUM(IF(Region='Oceania',Sales,0)) As Oceania,
        SUM(IF(Region='Canada',Sales,0)) As Canada,
        SUM(IF(Region='EMEA',Sales,0)) As EMEA,
        SUM(IF(Region='South East Asia',Sales,0)) As SE_Asia,
        SUM(IF(Region='Central',Sales,0)) As Central,
        SUM(IF(Region='East',Sales,0)) As East,
        SUM(IF(Region='West',Sales,0)) As West,
        SUM(IF(Region='North',Sales,0)) As North,
        SUM(IF(Region='South',Sales,0)) As South
    FROM praxisdb.fact43
        GROUP BY Segment WITH ROLLUP
    )
as sums
order by Dims desc
""")
CrossTab2.show()

+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|       Dims|Africa|North_Asia|Central_Asia|Caribbean|Oceania|Canada|  EMEA|SE_Asia|Central|  East|  West|  South|  Totals|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|Home Office|155077|    175177|      148235|    57395| 197812| 11896|148862|      0| 491961|127465|136736| 280016| 1930632|
|  Corporate|204948|    265762|      235552|   104535| 322831| 19319|250559|      0| 850405|200422|225870| 496029| 3176232|
|   Consumer|423751|    407410|      369052|   162351| 579564| 35717|406763|      0|1480033|350947|362908| 824915| 5403411|
|    #Totals|783776|    848349|      752839|   324281|1100207| 66932|806184|      0|2822399|678834|725514|1600960|10510275|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+



In [49]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))

ॐ श्री सरस्वत्यै नमः 2024-05-03 09:49:53.900292+05:30


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)